<a href="https://colab.research.google.com/github/kcezario/conhecimento_acessivel/blob/main/Projeto_Alura_Gemini_Acessibilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install -q -U SpeechRecognition
!pip install -q -U faster-whisper
!pip install -q -U wikipedia

In [2]:
import os
import re
import numpy as np
import sys
import pandas as pd
import google.generativeai as genai
import wikipedia
from faster_whisper import WhisperModel
from io import BytesIO
from base64 import b64decode
from google.colab import output
from google.colab import userdata
from IPython.display import Javascript

In [3]:
API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=API_KEY)

In [4]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

In [5]:
prompt_gemini = {
  'cumprimento': "Você é um assistente simpático, e o texto que você gerar será lido por uma tecnologia de text-to-speech. Cumprimente o usuário. Mantenha o texto curto, simpático e pergunte o que ele quer saber hoje.",
  'nao_entendi': "Você é um assistente simpático que utiliza text-to-speech. Diga ao usuário que você não entendeu o comando e peça para ele repetir de forma gentil.",
  'despedida': "Você é um assistente simpático que se comunica por text-to-speech. Despeça-se do usuário com um texto curto e gentil.",
  'resumo': "Você é um assistente simpático que usa text-to-speech. Pergunte ao usuário se ele gostaria de ouvir um resumo do artigo.",
  'leitura_completa': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário se ele gostaria de ouvir o artigo completo.",
  '3_perguntas':"Você é um assistente simpático que utiliza text-to-speech. Ofereça ao usuário três opções: ouvir um resumo do artigo, ouvir o artigo completo ou pesquisar outro assunto.",
  'esclarecimento': "Você é um assistente simpático que usa text-to-speech. Pergunte ao usuário se ele deseja que algum ponto específico do artigo seja esclarecido.",
  'perguntas': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário se ele gostaria de responder a algumas perguntas sobre o tema para testar seus conhecimentos."
}

In [6]:
gemini_model = genai.GenerativeModel("gemini-1.0-pro-latest")  # Modelo Gemini
gemini_conversa = gemini_model.start_chat(history=[])

In [7]:
def record(sec=3, conversa_iniciada=False):
  if conversa_iniciada:
    comando = 'nao_entendi'
  else:
    comando = 'cumprimento'
  gemini_assist = falar_gemini(comando)
  conversa_iniciada = True
  print(gemini_assist)
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Pensando...")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [8]:
# Função para transcrever áudio usando faster_whisper
def transcrever_audio(audio_bytes):
  with BytesIO(audio_bytes) as audio_stream:
    segments, info = whisper_model.transcribe(audio_stream, language="pt")
    texto = ' '.join([segment.text for segment in segments])
    return texto

In [9]:
# # Função para capturar áudio e detectar a palavra de ativação
# def capturar_e_detectar():
#   global listening_for_wake_word
#   listening_for_wake_word = True
#   while listening_for_wake_word:
#     audio_bytes = record(sec=3)  # Grava por 3 segundos
#     texto = transcrever_audio(audio_bytes)
#     texto = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', texto)
#     if WAKE_WORD in texto.lower():
#       print("Palavra de ativação detectada!")
#       listening_for_wake_word = False
#       return capturar_comando()
#     else:
#       print("Aguardando palavra de ativação...")

In [10]:
def executar_acao(intencao, comando):
    """Executa a ação correspondente à intenção do usuário."""
    global artigo_atual
    if intencao == 'não entendi':
        return falar_gemini('nao_entendi')
    elif intencao == 'pesquisar':
        tema_wikipedia = extrair_tema_wikipedia(comando)
        artigo_atual = pesquisar_wikipedia(tema_wikipedia)
        return falar_gemini('3_perguntas')
    elif intencao == 'sair':
        resposta = falar_gemini('despedida')
        print(resposta)
        exit()
    elif intencao == '3_perguntas':
        return artigo_atual  # Retorna o artigo armazenado
    else:
        return "Desculpe, não entendi o seu pedido."

In [11]:
def menu_principal():
  """Gerencia o menu principal do assistente."""
  df_intencoes['embedding'] = df_intencoes['conteudo'].apply(gerar_embedding) # Gera os embeddings ao entrar no menu
  while True:
    audio_bytes = record(sec=5)
    texto = transcrever_audio(audio_bytes)
    texto = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', texto).lower()
    print("Transcrição:", texto)

    if not texto or texto.isspace():
      print("Resposta do Gemini:", falar_gemini('nao_entendi'))
      continue

    intencao = classificar_intencao(texto, df_intencoes)
    print("Intenção:", intencao)

    if intencao == 'sair':
      break
    elif intencao == 'pesquisar':
      menu_pesquisa(texto)
    else:
      resposta = executar_acao(intencao, texto)
      print("Resposta do Gemini:", resposta)

In [12]:
def menu_pesquisa(comando):
  """Gerencia o menu de pesquisa de artigos."""
  global artigo_atual
  tema_wikipedia = extrair_tema_wikipedia(comando)
  artigo_atual = pesquisar_wikipedia(tema_wikipedia)
  print("Resposta do Gemini:", falar_gemini('3_perguntas'))

  df_3_perguntas['embedding'] = df_3_perguntas['conteudo'].apply(gerar_embedding) # Gera os embeddings ao entrar no menu
  while True:
    audio_bytes = record(sec=5)
    texto = transcrever_audio(audio_bytes)
    texto = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', texto).lower()
    print("Transcrição:", texto)

    if not texto or texto.isspace():
      print("Resposta do Gemini:", falar_gemini('nao_entendi'))
      continue

    intencao = classificar_intencao(texto, df_3_perguntas)
    print("Intenção:", intencao)

    if intencao == 'outro assunto':
      break

    resposta = executar_acao(intencao, texto)
    print("Resposta do Gemini:", resposta)

In [13]:
def capturar_e_detectar():
  """Inicia o assistente de voz."""
  print("Iniciando o assistente...")
  menu_principal()
  print("Terminando o programa...")

In [14]:
def capturar_comando():
  print("Fale seu comando...")
  audio_bytes = record(sec=5)  # Grava por 5 segundos
  comando = transcrever_audio(audio_bytes)
  resposta = obter_resposta_gemini(comando)
  print("Resposta do Gemini:", resposta)
  return resposta

In [15]:
# Função para enviar o comando para a API do Gemini e obter a resposta
def obter_resposta_gemini(comando):
  # Envie o comando e obtenha a resposta
  gemini_conversa.send_message(comando)
  resposta = gemini_conversa.last_text
  return resposta

In [16]:
def falar_gemini(chave_prompt, texto=""):
  prompt = prompt_gemini.get(chave_prompt)
  if not prompt:
    prompt = prompt_gemini.get(nao_entendi)
  resposta = gemini_conversa.send_message(prompt)
  retorno = resposta.text
  return retorno

In [17]:
model_embed = "models/embedding-001"

df_intencoes = pd.DataFrame({
    'titulo': ["não entendi", "pesquisar", "sair"],
    'conteudo': ["coisas sem sentido", "desejo que você pesquise, me fale sobre, quero saber, quero estudar", "quero sair, adeus, nos vemos, até logo"]
})

df_3_perguntas = pd.DataFrame({
    'titulo': ["resumo", "artigo completo", "outro assunto"],
    'conteudo': ["quero um resumo, resuma para mim, explique rapidamente", "leia tudo, quero saber mais, comece a leitura", "quero falar de outra coisa, quero entnder sobre outro assunto"]
})

def gerar_embedding(texto, model=model_embed):
  """Gera um embedding para o texto usando a API do Gemini."""
  return genai.embed_content(model=model, content=texto, task_type="RETRIEVAL_QUERY")["embedding"]

In [18]:
def classificar_intencao(consulta, df, model=model_embed):
  """Classifica a intenção do usuário com base na similaridade de embeddings."""
  embedding_da_consulta = gerar_embedding(consulta, model)
  produtos_escalares = np.dot(np.stack(df["embedding"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["titulo"]

In [19]:
def pesquisar_wikipedia(tema):
  """Pesquisa um tema na Wikipedia e retorna um resumo."""
  try:
    wikipedia.set_lang("pt")
    resultados = wikipedia.search(tema)
    if resultados:
      pagina = wikipedia.page(resultados[0])
      resposta = falar_gemini('resumo')
      resposta += f" Aqui está um resumo sobre {pagina.title}: {pagina.summary}"
      return resposta
    else:
      return "Desculpe, não encontrei nenhum artigo na Wikipédia sobre esse tema."
  except wikipedia.exceptions.DisambiguationError as e:
    return f"Sua pesquisa é ambígua. Poderia se referir a: {e.options}"
  except wikipedia.exceptions.PageError:
    return "Desculpe, não encontrei nenhum artigo na Wikipédia com esse título."


In [20]:
def extrair_tema_wikipedia(texto):
  """Pede ao Gemini para extrair o tema principal da frase para pesquisa na Wikipédia."""
  prompt = f"""Você é um assistente inteligente que ajuda a encontrar informações na Wikipédia.
  Analise a seguinte frase e identifique o assunto principal que o usuário deseja pesquisar: "{texto}".
  Forneça apenas o assunto principal, formatado como uma única frase, ideal para ser usado como consulta na API da Wikipédia.
  Por exemplo, se a frase for "Gostaria de saber mais sobre a história da pizza", a resposta deve ser "História da pizza".
  Sua resposta:"""

  resposta = gemini_conversa.send_message(prompt)
  tema_wikipedia = resposta.text
  return tema_wikipedia


In [21]:
# Carregar modelo Whisper para reconhecimento de voz
whisper_size = 'base'
whisper_model = WhisperModel(whisper_size, device="cpu", compute_type="int8")

print("Configuração e autenticação concluídas!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Configuração e autenticação concluídas!


In [23]:
# Iniciar a captura de áudio e detecção da palavra de ativação

capturar_e_detectar()
print("Fase 3 concluída!")

Iniciando o assistente...
Olá! Prazer em conhecê-lo. O que você gostaria de saber hoje?


<IPython.core.display.Javascript object>

Pensando...
Transcrição:  me fale sobre a segunda guerra mundial
Intenção: pesquisar
Resposta do Gemini: Você gostaria de:

1. Ouvir um resumo do artigo
2. Ouvir o artigo completo
3. Pesquisar outro assunto
Olá! Como posso ajudar?


<IPython.core.display.Javascript object>

KeyboardInterrupt: 